In [1]:
# In this notebook we create a classifier of wine quality using as predictor variables the number of words
# and polarity of the sentiment analysis
from __future__ import unicode_literals
from textblob import TextBlob
import pandas as pd
import nltk

In [2]:
!pwd

/Users/luis/Desktop/tfm_folder


In [3]:
wine = pd.read_csv('/Users/luis/Desktop/tfm_folder/winemag-data_first150k.csv.zip',sep=",")

In [4]:
wine.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [5]:
wine = wine.drop_duplicates()

In [6]:
wine= wine.dropna()

SENTIMENT ANALYSIS

In [7]:
first_description = wine.description[0]
print(first_description)
opinion_wine = TextBlob(first_description.decode('unicode-escape'))
print(opinion_wine.sentiment.polarity)
print(opinion_wine.sentiment.subjectivity)

This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.
0.224074074074
0.666666666667


In [8]:
def polarity_function(review):#measures polarity of wine description
    opinion_wine=TextBlob(review.decode('unicode-escape'))
    return opinion_wine.sentiment.polarity

def subjectivity_function(review): #measures subjectivity of wine description
    opinion_wine=TextBlob(review.decode('unicode-escape'))
    return opinion_wine.sentiment.subjectivity

def words_function(review): # measures the length of the wine description by number of words
    t=TextBlob(review.decode('unicode-escape'))
    return len(t.words)

In [9]:
wine['polarity']= wine.description.apply(polarity_function)

In [10]:
wine['num_words'] = wine.description.apply(words_function)

In [11]:
def rating_type(score):
    if score > 92:
        return 1
    if score <= 84:
        return 0
    if score>84 and score<=92:
        return 2


wine['quality'] = wine.points.apply(rating_type)

In [12]:
wine['quality'].value_counts()

2    28612
0     5763
1     4866
Name: quality, dtype: int64

In [13]:
wine_f = wine[['num_words','polarity','quality']]

In [14]:
wine_f = wine_f[wine_f.quality.isin([0,1])==True]

In [16]:
wine_f.head()

,num_words,polarity,quality
0,60,0.224074,1
2,47,0.137500,1
3,63,0.105195,1
8,43,0.300000,1
9,45,0.212000,1
